In [2]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm
import pandas as pd
from  easydict import EasyDict

In [3]:
import torch
from sklearn.metrics import accuracy_score
import argparse

from predictor import Predictor
from datasets import *
from utils import *
from models import simple_NN

## 각각의 test frame을 추론하고, 50개(1 video = 50frame) frame을 평균내서 가장 높은 값을 가진 클래스를 할당 하는 것

In [15]:
from  easydict import EasyDict
args = EasyDict({
    "BATCH_SIZE" : 32,
    "MODEL_NAME" : 'efficientnet_b0',
    "ENSEMBLE" : None,
    "IMG_PATH" : "./data/div_2width_train/*",
    "CSV_PATH" : "./data/new_test.csv",
    "OUTPUT" : "./results/div_2width_effib0_224.csv",
    "CHECKPOINT" : ["./ckpt/div_2width_effib0_224/1E-val1.0-efficientnet_b0.pth"]
})

In [16]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = simple_NN(args.MODEL_NAME, num_classes=13).to(device)
predictor = Predictor(model, device, args)



MODEL LOAD ... from ./ckpt/div_2width_effib0_224/1E-val1.0-efficientnet_b0.pth


In [17]:
preds, pred_inds = predictor.run()

100%|██████████████████████████████████████████████████████████████████████████████████████| 2813/2813 [13:06<00:00,  3.58it/s]


In [ ]:
df = pd.DataFrame({'pred_value' : preds, 'pred_index' : pred_inds})

In [ ]:
pred_list = [0 for i in range(1800)]
cnt = 0
for j in tqdm(range(0, 90000, 50)) :    
    best_score = 0
    best_cls = 0
    for i in range(13) :
        mean_score = sum(df['pred_value'][j : j + 49][df['pred_index'] == i].values)/(len(df['pred_value'][j : j + 50][df['pred_index'] == i].values.tolist()) + +0.000001)
        if mean_score >= best_score  :
            best_score = mean_score
            best_cls = i

    pred_list[cnt] = best_cls
    cnt += 1

In [46]:
df = pd.read_csv("./data/sample_submission.csv")
df['label'] = pred_list

In [51]:
df.to_csv('./results/div_2width_effib0_224.csv', index = False)

## 이미지 channel 단위로 겹치기

In [1]:
import cv2
import os
from glob import glob
import numpy as np
import pandas as pd

import albumentations as A
from albumentations.pytorch import ToTensorV2
# from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import Dataset, DataLoader

In [2]:
def collate_fn(batch):
    img_stack = []
    label_stack = []
    _transforms = A.Compose([
        A.Resize(384, 384),
        # A.Resize(224, 224),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False,
                    p=1.0),
    ])
    print(batch[:][0].shape)
    for img, label in batch:
        # img 나누기
#         w_ratio = int(img.shape[1] / 16)
#         h_ratio = int(img.shape[0] / 8) # 16

        label_stack.append(label)
        label_stack.append(img)
#         for w in range(16):
#             for h in range(8): # 16
#                 cropped_img = _transforms(image=img[h * h_ratio: (h + 1) * h_ratio, w * w_ratio: (w + 1) * w_ratio])['image'].transpose(2, 0, 1)
#                 img_stack.append(cropped_img.tolist())

    return torch.Tensor(img_stack), torch.Tensor(label_stack)

In [3]:
class _ChannelStackDataset(Dataset):
    def __init__(self, _df, labels, transform=None):
        self._df = _df
        self.labels = labels

        self.transform = transform

    def __len__(self):
        return len(self._df)

    def __getitem__(self, index):
        img_path = self._df['img_path'].iloc[index]
        
        # print(img_path)
        # image = cv2.imread(os.path.join('./data', *img_path.split('/')[2:]))
        image = cv2.imread(img_path)

        if self.transform :
            image = self.transform(image=image)['image']
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.labels is not None:
            label = self.labels[index]
            return image, label

        else:
            return image

In [ ]:
# if __name__ == '__main__':
csv_path = "./data/div_2width_train.csv"

df = pd.read_csv(csv_path)
label_set = df['label']
transform = A.Compose([
                A.Resize(224, 224),
                # A.Rotate(limit=(45), p=1),
                # A.RandomGridShuffle(p=grid_shuffle_p, grid=(2,2)),
                A.Normalize(),
                ToTensorV2()
            ])
ds = _ChannelStackDataset(df, label_set, transform)
dl = DataLoader(ds, batch_size=32, shuffle=False, num_workers=1)#, collate_fn=collate_fn)

for img, label in dl :
    print(img.shape)
    print(label.shape)
    break

In [5]:
ds

In [6]:
dl

In [8]:
for img, label in dl :
    print(img.shape)
    print(label.shape)
    break

NameError: name 'dl' is not defined

## timing, weather, ego-involve + crash 학습용 csv 만들기

### timing
- night = 0
- day = 1

In [36]:
path = "./data/50frame_train.csv"
df = pd.read_csv(path)
_df = df[df['label'] != 0]

new_label = [1 if label % 2 == 1 else 0 for label in _df['label'] ]
print("day : ",new_label.count(1))
print("night : ",new_label.count(0))

day :  40400
night :  5350


In [40]:
_df['label'] = new_label

path = "./data/50f_timing_train.csv"
_df.to_csv(path, index=False)

C:\Users\quhb2\AppData\Local\Temp\ipykernel_42468\3172057110.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _df['label'] = new_label


In [132]:
day =  40400
night =  5350
z = [5350, 40400]
np.max(z)/np.array(z)

array([7.55140187, 1.        ])

### weather
- normal = 0
- snowy = 1
- Rainy = 2

In [47]:
path = "./data/50frame_train.csv"
df = pd.read_csv(path)

_df = df[df['label'] != 0]

new_label = []
for label in _df['label'] : 
    
    # normal
    if label in [1,2,7,8] :
        new_label.append(0)
    
    # swnoy
    elif label in [3,4,9,10] :
        new_label.append(1)
    
    # rainy
    elif label in [5,6,11,12] :
        new_label.append(2)
        

print("normal : ",new_label.count(0))
print("swnoy : ",new_label.count(1))
print("rainy : ",new_label.count(2))

normal :  35800
swnoy :  6450
rainy :  3500


In [51]:
_df['label'] = new_label

path = "./data/50f_weather_train.csv"
_df.to_csv(path, index=False)

C:\Users\quhb2\AppData\Local\Temp\ipykernel_42468\279117756.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _df['label'] = new_label


In [143]:
normal =  35800
swnoy =  6450
rainy =  3500
z = [normal, swnoy, rainy]
np.max(z)/np.array(z)

array([ 1.        ,  5.5503876 , 10.22857143])

### ego-involve + crash 
- No ego + No crash = 0
- No ego + crash = 1
- ego + cash= 2

In [53]:
path = "./data/50frame_train.csv"
df = pd.read_csv(path)

new_label = []
for label in df['label'] :
    # No ego + No crash
    if label == 0 :
        new_label.append(0)
    
    #No ego + crash = 1
    elif label in [7, 8, 9, 10, 11, 12] : 
        new_label.append(1)
    
    # ego + cash = 2
    elif label in [1, 2, 3, 4, 5, 6] :
        new_label.append(2)
    
print("No ego + No crash : ",new_label.count(0))
print("No ego + crash : ",new_label.count(1))
print("ego + cash : ",new_label.count(2))

No ego + No crash :  89150
No ego + crash :  21200
ego + cash :  24550


In [54]:
df['label'] = new_label

path = "./data/50f_EgoCrash_train.csv"
df.to_csv(path, index=False)

In [61]:
import torch
import torch.nn as nn

In [145]:
class FocalLoss(nn.Module) :
    def __init__(self, alpha=2, gamma=2, logits=False, reduction='none') :
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduction = reduction

    def forward(self, inputs, targets) :
        ce_loss = nn.CrossEntropyLoss(reduction=self.reduction)(inputs, targets)
        pt = torch.exp(-ce_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * ce_loss

        if self.reduction :
            return torch.mean(F_loss)
        else :
            return F_loss
        
crit = FocalLoss()
# crit = nn.BCEWithLogitsLoss().to(dtype=torch.float)

In [199]:
# a= torch.tensor([1, 0, 0, 0, 0, 1, 1, 1], dtype=torch.float)
a =  torch.tensor([1, 1,  1, 0], dtype=torch.float)
a.shape

torch.Size([4])

In [200]:
b= torch.tensor([[1, 0], [1, 0], [0, 1],[0, 1]], dtype=torch.float)
b.shape

torch.Size([4, 2])

## Ego + crash 모자이크 이미지 만들기

In [1]:
path = "./data/50f_EgoCrash_train.csv"
df = pd.read_csv(path)

NameError: name 'pd' is not defined